In [4]:
import torchvision
from torchvision import transforms
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch

image_path = './'
transform = transforms.Compose([
    transforms.ToTensor()
    ])
mnist_train_dataset = torchvision.datasets.MNIST(
    root=image_path, train=True,
    transform=transform, download=True
    )
mnist_test_dataset = torchvision.datasets.MNIST(
    root=image_path, train=False,
    transform=transform, download=True
    )
batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset,
                                          batch_size, shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 19.7MB/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 627kB/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.88MB/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 3.85MB/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [5]:
hidden_units = [32, 16]
image_size = mnist_train_dataset[0][0].shape
input_size = image_size[0] * image_size[1] * image_size[2]
all_layers = [nn.Flatten()]
for hidden_unit in hidden_units:
  layer = nn.Linear(input_size, hidden_unit)
  all_layers.append(layer)
  all_layers.append(nn.ReLU())
  input_size = hidden_unit
all_layers.append(nn.Linear(hidden_units[-1], 10))
model = nn.Sequential(*all_layers)
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=32, bias=True)
  (2): ReLU()
  (3): Linear(in_features=32, out_features=16, bias=True)
  (4): ReLU()
  (5): Linear(in_features=16, out_features=10, bias=True)
)

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
torch.manual_seed(1)
num_epochs = 20
for epoch in range(num_epochs):
  accuracy_hist_train = 0
  for x_batch, y_batch in train_dl:
    pred = model(x_batch)
    loss = loss_fn(pred, y_batch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    is_correct = (
        torch.argmax(pred, dim=1) == y_batch
        ).float()
    accuracy_hist_train += is_correct.sum()
  accuracy_hist_train /= len(train_dl.dataset)
  print(f'Epoch {epoch} Accuracy '
  f'{accuracy_hist_train:.4f}')

Epoch 0 Accuracy 0.9755
Epoch 1 Accuracy 0.9778
Epoch 2 Accuracy 0.9783
Epoch 3 Accuracy 0.9803
Epoch 4 Accuracy 0.9804
Epoch 5 Accuracy 0.9815
Epoch 6 Accuracy 0.9820
Epoch 7 Accuracy 0.9827
Epoch 8 Accuracy 0.9835
Epoch 9 Accuracy 0.9840
Epoch 10 Accuracy 0.9844
Epoch 11 Accuracy 0.9853
Epoch 12 Accuracy 0.9856
Epoch 13 Accuracy 0.9860
Epoch 14 Accuracy 0.9866
Epoch 15 Accuracy 0.9883
Epoch 16 Accuracy 0.9876
Epoch 17 Accuracy 0.9888
Epoch 18 Accuracy 0.9890
Epoch 19 Accuracy 0.9889


In [9]:
pred = model(mnist_test_dataset.data / 255.)
is_correct = (
    torch.argmax(pred, dim=1) ==
    mnist_test_dataset.targets
    ).float()
print(f'Test accuracy: {is_correct.mean():.4f}')

Test accuracy: 0.9639
